In [2]:
import pandas as pd

#Data set: https://www.kaggle.com/datasets/hiraahmed/movielense20m

In [3]:
# Data set containing movieId, movie title, and movie genre information
df_m = pd.read_csv(r"C:\Users\utku\Desktop\miuul\HybridRecommender\datasets\movie.csv")
# Data set containing UserID, movie title, rating given to the movie, and timestamp information
df_r = pd.read_csv(r"C:\Users\utku\Desktop\miuul\HybridRecommender\datasets\rating.csv.")

In [4]:
# Only the IDs of the movies that users voted on are present in the Ratings dataset.
# We are adding the movie titles and genres corresponding to the IDs from the Movie dataset.

df = df_r.merge(df_m, on='movieId', how='left')

In [5]:
# Calculating the total number of votes cast for each movie.

df["title"].value_counts().head()
comment_counts = pd.DataFrame(df["title"].value_counts())

In [6]:
# Finding movies with less than or equal to 1000 votes.
rare_movies = comment_counts[comment_counts["count"] <= 1000].index

# Selecting movies that are not in the rare_movies list.
common_movies = df[~df["title"].isin(rare_movies)]


In [7]:
# Creating a pivot table for the DataFrame where the index represents userIDs,
# columns represent movie titles, and values represent ratings.
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
user_movie_df

In [13]:
# Function to create a user-movie DataFrame with operations encapsulated.
#def create_user_movie_df():
    #import pandas as pd
    #df_m = pd.read_csv(r"C:\Users\utku\Desktop\miuul\HybridRecommender\datasets\movie.csv")
    #df_r = pd.read_csv(r"C:\Users\utku\Desktop\miuul\HybridRecommender\datasets\rating.csv.")
    #df = df_r.merge(df_m, on='movieId', how='left')
    #comment_counts = pd.DataFrame(df["title"].value_counts())
    #rare_movies = comment_counts[comment_counts["count"] <= 1000].index
    #common_movies = df[~df["title"].isin(rare_movies)]
    #user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    #return user_movie_df

#user_movie_df = create_user_movie_df()

In [14]:
# Choosing random user id
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=45).values)
# random_user=28941

C:\Users\utku\AppData\Local\Temp\ipykernel_27228\627965561.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=45).values)


In [19]:
# Create a new DataFrame named random_user_df consisting of observation units for the selected user.
random_user_df = user_movie_df[user_movie_df.index == random_user]
random_user_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
28941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Assign the movies that the selected user voted on to a list named movies_watched.
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
movies_watched

['Ace Ventura: Pet Detective (1994)',
 'Ace Ventura: When Nature Calls (1995)',
 'Aladdin (1992)',
 'American President, The (1995)',
 'Apollo 13 (1995)',
 'Babe (1995)',
 'Bullets Over Broadway (1994)',
 'Clueless (1995)',
 'Disclosure (1994)',
 'Forrest Gump (1994)',
 'Four Weddings and a Funeral (1994)',
 'Home Alone (1990)',
 'Jurassic Park (1993)',
 'Like Water for Chocolate (Como agua para chocolate) (1992)',
 'Little Women (1994)',
 "Mr. Holland's Opus (1995)",
 'Mrs. Doubtfire (1993)',
 'Much Ado About Nothing (1993)',
 "Muriel's Wedding (1994)",
 'Nine Months (1995)',
 'Operation Dumbo Drop (1995)',
 'Piano, The (1993)',
 'Postman, The (Postino, Il) (1994)',
 'Ready to Wear (Pret-A-Porter) (1994)',
 'Remains of the Day, The (1993)',
 'Sabrina (1995)',
 "Schindler's List (1993)",
 'Secret Garden, The (1993)',
 'Sense and Sensibility (1995)',
 'Shadowlands (1993)',
 'Silence of the Lambs, The (1991)',
 'Star Trek: Generations (1994)',
 'Stargate (1994)']

In [20]:
# Select columns corresponding to the movies watched by the selected user from user_movie_df and create a new dataframe named movies_watched_df.

movies_watched_df = user_movie_df[movies_watched]


In [21]:
# Create a new dataframe named user_movie_count containing information about how many movies each user watched from the movies watched by the selected user.

user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]
user_movie_count[user_movie_count["movie_count"] > 20].sort_values("movie_count", ascending=False).head(20)
user_movie_count[user_movie_count["movie_count"] == 33].count()

userId         17
movie_count    17
dtype: int64

In [22]:
# Users who have watched 60% or more of the movies voted by the selected user are considered similar users.
# Create a list named users_same_movies from the IDs of these users.

perc = len(movies_watched) * 60 / 100
users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]


In [24]:
#Filter the movies_watched_df dataframe to find the IDs of users similar to the selected user within the user_same_movies list.

df_final = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)], random_user_df[movies_watched]])
df_final

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Aladdin (1992),"American President, The (1995)",Apollo 13 (1995),Babe (1995),Bullets Over Broadway (1994),Clueless (1995),Disclosure (1994),Forrest Gump (1994),...,Ready to Wear (Pret-A-Porter) (1994),"Remains of the Day, The (1993)",Sabrina (1995),Schindler's List (1993),"Secret Garden, The (1993)",Sense and Sensibility (1995),Shadowlands (1993),"Silence of the Lambs, The (1991)",Star Trek: Generations (1994),Stargate (1994)
userId,,,,,,,,,,,,,,,,,,,,,
91,2.5,2.0,NaN,4.0,3.5,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,2.5,4.5,NaN,NaN,NaN,3.0,3.5,4.0
130,4.0,3.0,NaN,3.0,3.0,NaN,NaN,3.0,5.0,5.0,...,NaN,3.0,NaN,5.0,NaN,NaN,3.0,5.0,NaN,3.0
156,3.0,NaN,NaN,5.0,5.0,3.0,NaN,NaN,4.0,5.0,...,NaN,NaN,4.0,5.0,NaN,4.0,4.0,5.0,3.0,4.0
158,2.0,1.0,4.0,4.0,3.0,5.0,NaN,4.0,NaN,5.0,...,NaN,5.0,3.0,5.0,5.0,4.0,5.0,5.0,NaN,NaN
160,NaN,NaN,3.0,5.0,4.0,4.0,NaN,3.0,2.0,5.0,...,NaN,5.0,NaN,5.0,NaN,NaN,5.0,4.0,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138279,3.0,NaN,NaN,3.0,5.0,5.0,5.0,4.0,NaN,5.0,...,NaN,5.0,NaN,5.0,NaN,4.0,NaN,5.0,3.0,3.0
138382,1.0,1.0,4.0,2.0,3.0,5.0,NaN,4.0,3.0,5.0,...,NaN,NaN,3.0,NaN,3.0,3.0,NaN,5.0,NaN,4.0
138415,1.0,NaN,3.0,4.0,3.0,5.0,NaN,NaN,NaN,4.0,...,NaN,5.0,NaN,5.0,3.0,4.0,4.0,NaN,3.0,NaN


In [25]:
# Create a new dataframe named corr_df containing correlations between users.

corr_matrix = df_final.T.corr()
corr_df = corr_matrix.reset_index().unstack()
corr_df = pd.DataFrame(corr_df, columns=['corr'])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df.reset_index(inplace=True)


In [27]:
# Filter users with high correlation (above 0.65) with the selected user and create a new dataframe named top_users.

top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][
    ["user_id_2", "corr"]].reset_index(drop=True)

top_users = top_users.sort_values(by='corr', ascending=False)

top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

top_users

,userId,corr
131,4139,1.000000
65,4139,1.000000
80,905,1.000000
14,905,1.000000
9,399,0.802181
...,...,...
35,2523,0.655212
81,1074,0.652730
15,1074,0.652730
44,3207,0.651830


In [29]:
# Merge top_users dataframe with the rating dataset.

top_users_ratings = top_users.merge(df_r[["userId", "movieId", "rating"]], how='inner')
top_users_ratings

,userId,corr,movieId,rating
0,4139,1.00000,150,4.0
1,4139,1.00000,153,3.0
2,4139,1.00000,161,5.0
3,4139,1.00000,165,4.0
4,4139,1.00000,185,4.0
...,...,...,...,...
15737,3207,0.65183,4343,4.0
15738,3207,0.65183,4344,4.0
15739,3207,0.65183,4351,3.0
15740,3207,0.65183,4367,4.0


In [31]:
# Create a new variable named weighted_rating consisting of the product of the corr and rating values for each user.

top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings

,userId,corr,movieId,rating,weighted_rating
0,4139,1.00000,150,4.0,4.00000
1,4139,1.00000,153,3.0,3.00000
2,4139,1.00000,161,5.0,5.00000
3,4139,1.00000,165,4.0,4.00000
4,4139,1.00000,185,4.0,4.00000
...,...,...,...,...,...
15737,3207,0.65183,4343,4.0,2.60732
15738,3207,0.65183,4344,4.0,2.60732
15739,3207,0.65183,4351,3.0,1.95549
15740,3207,0.65183,4367,4.0,2.60732


In [32]:
# Create a new dataframe named recommendation_df containing the average weighted ratings of all users for each movie ID.

recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"}).reset_index()

In [34]:
# Select movies with a weighted rating greater than 3.5 in recommendation_df and sort them by weighted rating.
# Save the first 5 observations as movies_to_be_recommend.

movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values("weighted_rating", ascending=False).head()
movies_to_be_recommend


,movieId,weighted_rating
751,1488,4.003746
2545,54881,3.952023
1495,3307,3.819625
30,35,3.794867
1172,2442,3.794867


In [37]:
# First 5 movies' names
movies_to_be_recommend.merge(df_m[["movieId", "title"]])



,movieId,weighted_rating,title
0,1488,4.003746,"Devil's Own, The (1997)"
1,54881,3.952023,"King of Kong, The (2007)"
2,3307,3.819625,City Lights (1931)
3,35,3.794867,Carrington (1995)
4,2442,3.794867,Hilary and Jackie (1998)


**Item-Based Recommendation**

In [38]:
# Make an item-based recommendation based on the title of the movie that the user watched last and rated the highest. Sample user
user = 108170


In [42]:
# Get the ID of the movie that the user rated 5 points and has the most recent rating.

user_ratings = df_r[df_r['userId'] == user]

target_movie = user_ratings.sort_values(['rating', 'timestamp'], ascending=[False, False]).iloc[0]

target_movie_id = target_movie['movieId']
target_movie_name = df_m[df_m['movieId'] == target_movie_id]['title'].values[0]
target_movie_name


'Wild at Heart (1990)'

In [41]:
# Filter the user_movie_df dataframe based on the selected movie ID in the user-based recommendation section.

user_movie_df = user_movie_df[user_movie_df[target_movie_name].notna()]
user_movie_df

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),...,Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zulu (1964),[REC] (2007),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN,2.0
424,NaN,NaN,3.5,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.5,NaN
455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
664,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137683,NaN,NaN,NaN,NaN,2.5,NaN,4.0,4.0,NaN,4.0,...,NaN,NaN,3.5,NaN,2.5,NaN,NaN,NaN,NaN,NaN
137839,NaN,3.5,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,...,NaN,NaN,3.5,NaN,NaN,NaN,NaN,3.5,NaN,0.5
138206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Using the filtered dataframe, find and sort the correlation of other movies with the selected movie.

target_movie_ratings = user_movie_df[target_movie_name]
correlations = user_movie_df.corrwith(target_movie_ratings).sort_values(ascending=False)
correlations

title
Wild at Heart (1990)                     1.000000
My Science Project (1985)                0.570187
Mediterraneo (1991)                      0.538868
Old Man and the Sea, The (1958)          0.536192
National Lampoon's Senior Trip (1995)    0.533029
                                           ...   
Evan Almighty (2007)                    -0.191395
Corrina, Corrina (1994)                 -0.224996
27 Dresses (2008)                       -0.228704
Baby Geniuses (1999)                    -0.288270
Titanic (1953)                          -0.376434
Length: 3159, dtype: float64

In [46]:
# Provide the top 5 recommendations excluding the selected movie itself.

top_recommendations = correlations.drop(target_movie_name).head(5)
top_recommendations

title
My Science Project (1985)                0.570187
Mediterraneo (1991)                      0.538868
Old Man and the Sea, The (1958)          0.536192
National Lampoon's Senior Trip (1995)    0.533029
Clockwatchers (1997)                     0.483337
dtype: float64